In [2]:
import telebot
import sqlite3

from threading import Thread
from time import sleep

def time_out (message):
    sleep (7)
    bot.delete_message (message.chat.id, message.id)

bot = telebot.TeleBot ('5978837391:AAFj2jm3yY3_YUH7aPhFeNtTv-PdNPi941A')
service = None
login = None

@bot.message_handler (commands = ['set'])
def user_set (message):
    conn = sqlite3.connect ('table.sql')
    cur = conn.cursor ()
    
    cur.execute ('CREATE TABLE IF NOT EXISTS users (id int PRIMARY KEY, service varchar (50), login varchar (50), password varchar (50))')

    conn.commit ()
    cur.close ()
    conn.close ()
    
    bot.send_message (message.chat.id, 'Введите название сервиса: ')
    bot.register_next_step_handler (message, user_service)
    
def user_service (message):
    global service
    service = message.text.strip ()
    
    conn = sqlite3.connect ('table.sql')
    cur = conn.cursor ()
    
    cur.execute ("SELECT service FROM users WHERE service = ? AND id = ?", [service, message.from_user.id])
    if cur.fetchone () != None:
        bot.send_message (message.chat.id, 'Для данного сервиса пароль и логин уже заданы')
    else:
        bot.send_message (message.chat.id, 'Введите логин: ')
        bot.register_next_step_handler (message, user_login)
    
    cur.close()
    conn.close()
    
def user_login (message):
    global login 
    login = message.text.strip ()
    
    bot.send_message (message.chat.id, 'Введите пароль: ')
    bot.register_next_step_handler (message, user_password)
    
def user_password (message):
    password = message.text.strip ()
    
    t1 = Thread (target = time_out, args = (message, ), daemon = True)
    t1.start ()
    
    ID = message.from_user.id
    
    conn = sqlite3.connect ('table.sql')
    cur = conn.cursor ()
    
    cur.execute ("INSERT INTO users (ID, service, login, password) VALUES ('%d', '%s', '%s', '%s')" %(ID, service, login, password))

    conn.commit ()
    cur.close ()
    conn.close ()

    bot.send_message (message.chat.id, 'Пользователь зарегистрирован!')
    
    t1.join ()
    
@bot.message_handler (commands = ['get'])
def user_get (message):
    bot.send_message (message.chat.id, 'Введите название сервиса, от которого хотите получить логин и пароль: ')
    bot.register_next_step_handler (message, output_log_and_pass)
       
def output_log_and_pass (message):
    get_service = message.text.strip ()
    
    conn = sqlite3.connect ('table.sql')
    cur = conn.cursor ()
    
    cur.execute ("SELECT service FROM users WHERE service = ? AND id = ?", [get_service, message.from_user.id])
    if cur.fetchone () is None:
        bot.send_message (message.chat.id, 'Сервис не найден')
    else:
        get_login = cur.execute ("SELECT login FROM users WHERE service = ? AND id = ?", [get_service, message.from_user.id]).fetchone()[0]
        get_password = cur.execute ("SELECT password FROM users WHERE service = ? AND id = ?", [get_service, message.from_user.id]).fetchone()[0]
        info = f'Логин: {get_login}, Пароль: {get_password}'
        bot.send_message (message.chat.id, info)
        
    cur.close()
    conn.close()
    
@bot.message_handler (commands = ['del'])
def user_del (message):
    bot.send_message (message.chat.id, 'Введите название сервиса, от которого хотите удалить логин и пароль: ')
    bot.register_next_step_handler (message, del_string_of_table)

def del_string_of_table (message):
    del_service = message.text.strip ()
    
    conn = sqlite3.connect ('table.sql')
    cur = conn.cursor ()
    
    cur.execute ("SELECT service FROM users WHERE service = ? AND id = ?", [del_service, message.from_user.id])
    if cur.fetchone () is None:
        bot.send_message (message.chat.id, 'Сервис не найден')
    else:
        cur.execute ('DELETE FROM users WHERE service = ? AND id = ?', [del_service, message.from_user.id])
        conn.commit ()
        bot.send_message (message.chat.id, 'Значения для сервиса успешно удалены')
    
    cur.close()
    conn.close()     

bot.polling (none_stop = True)